# NLP with Transformers

I have used this notebook :https://www.kaggle.com/tuckerarrants/bert-with-huggingface-transformers/notebook as a refernece for making my model 


In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
import numpy as np
import re
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku

In [2]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")

In [3]:
test_id = test['id']

In [4]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Data preprocessing

Here we do some text preprocessing and drop all the columns that are not required by us 

In [5]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [6]:
train['text']=train['text'].apply(lambda x : remove_URL(x))

In [7]:
train['text']=train['text'].apply(lambda x : remove_html(x))

In [8]:
train['text']=train['text'].apply(lambda x: remove_emoji(x))

In [9]:
train['text']=train['text'].apply(lambda x : remove_punct(x))

In [10]:
train = train.drop(columns = {'keyword', 'location', 'id'})

This is how our data looks like after preprocessing

In [11]:
train.head()

,text,target
0,Our Deeds are the Reason of this earthquake Ma...,1
1,Forest fire near La Ronge Sask Canada,1
2,All residents asked to shelter in place are be...,1
3,13000 people receive wildfires evacuation orde...,1
4,Just got sent this photo from Ruby Alaska as s...,1


In [12]:
test = test.drop(columns = {'id', 'location', 'keyword'})

In [13]:
test.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


Now let's finally import our model and tokenizer

In [14]:
from transformers import TFBertModel, BertModel
from transformers import BertTokenizer

In [15]:
bert_base = TFBertModel.from_pretrained('bert-base-uncased')
TOKENIZER = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Tokenize

Let's define the function to tokenize our train and test data 

In [16]:
def bert_encode(data,maximum_len) :
    input_ids = []
    attention_masks = []
  

    for i in range(len(data.text)):
        encoded = TOKENIZER.encode_plus(data.text[i],
                                        add_special_tokens=True,
                                        max_length=maximum_len,
                                        pad_to_max_length=True,
                                        return_attention_mask=True)
      
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

# Hyperparameters

Lets set the values for our hyperparameters that we will be using 

In [17]:
BATCH_SIZE = 16


EPOCHS = 2

#we will not be using metadata 
USE_META = False

ADD_DENSE = False
DENSE_DIM = 64

ADD_DROPOUT = True
DROPOUT = .2

TRAIN_BASE = True

In [18]:
def build_model(model_layer, learning_rate, use_meta = USE_META, add_dense = ADD_DENSE,
               dense_dim = DENSE_DIM, add_dropout = ADD_DROPOUT, dropout = DROPOUT):
    
    
    input_ids = tf.keras.Input(shape=(60,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(60,),dtype='int32')
    
    
    
    transformer_layer = model_layer([input_ids,attention_masks])
    
    
    output = transformer_layer[1]
    
        
    if add_dense:
        print("Training with additional dense layer...")
        output = tf.keras.layers.Dense(dense_dim,activation='relu')(output)
    
    
    if add_dropout:
        print("Training with dropout...")
        output = tf.keras.layers.Dropout(dropout)(output)
    
    
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    
    print("Training without meta-data...")
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)

    model.compile(tf.keras.optimizers.Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
print('Encoding Tweets...')
train_input_ids,train_attention_masks = bert_encode(train,60)
test_input_ids,test_attention_masks = bert_encode(test,60)
print('Tweets encoded')
print('')

#print('Train length:', len(train_input_ids))
#print('Test length:', len(test_input_ids))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Encoding Tweets...


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Tweets encoded



In [20]:
BERT_base = build_model(bert_base, learning_rate = 1e-5)
BERT_base.summary()

Training with dropout...
Training without meta-data...
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 

In [21]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('base_model.h5', monitor='val_loss', save_best_only = True, save_weights_only = True)

# Train the model

Finally let's train our model

In [22]:
history = BERT_base.fit([train_input_ids,train_attention_masks], train.target, validation_split = .2, epochs = EPOCHS, callbacks = [checkpoint], batch_size = BATCH_SIZE)

Epoch 1/2
381/381 [==============================] - 80s 168ms/step - loss: 0.5307 - accuracy: 0.7404 - val_loss: 0.3918 - val_accuracy: 0.8260
Epoch 2/2
381/381 [==============================] - 61s 161ms/step - loss: 0.3586 - accuracy: 0.8545 - val_loss: 0.3788 - val_accuracy: 0.8411


# Predictions

In [23]:
 preds_base = BERT_base.predict([test_input_ids,test_attention_masks])

In [24]:
submission = pd.DataFrame()
submission['id'] = test_id
submission['prob'] = preds_base
submission['target'] = np.round(submission['prob']).astype(int)
submission.head()

,id,prob,target
0,0,0.875268,1
1,2,0.937326,1
2,3,0.951625,1
3,9,0.976114,1
4,11,0.983409,1
